In [1]:
from keras.models import load_model
from tensorflow.keras.layers import Layer
from transformers import TFBertModel

# Завантажуємо модель із врахуванням кастомного шару
model = load_model('model.h5')

2024-12-29 02:01:49.759480: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 AVX512F AVX512_VNNI FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


ValueError: Unknown layer: 'BertLayer'. Please ensure you are using a `keras.utils.custom_object_scope` and that this object is included in the scope. See https://www.tensorflow.org/guide/keras/save_and_serialize#registering_the_custom_object for details.

In [6]:
import tensorflow as tf
# Кастомний шар для інтеграції з BERT
class BertLayer(Layer):
    def __init__(self, pretrained_model_name="bert-base-uncased", trainable=False, **kwargs):
        super(BertLayer, self).__init__(**kwargs)
        # Завантажуємо попередньо навчений BERT
        self.bert = TFBertModel.from_pretrained(pretrained_model_name)
        self.bert.trainable = trainable  # Заморожуємо або розморожуємо шари залежно від параметра trainable

    def call(self, inputs):
        # Вхідні дані: input_ids та attention_mask
        input_ids, attention_mask = inputs
        # Передаємо дані через BERT
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state  # Повертаємо тільки last_hidden_state

custom_objects = {'BertLayer': BertLayer}

# Завантаження моделі з кастомним шаром
try:
    model = tf.keras.models.load_model('model.h5', custom_objects=custom_objects)
    print("Модель успішно завантажена!")
except Exception as e:
    print(f"Помилка завантаження моделі: {e}")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

Помилка завантаження моделі: string indices must be integers, not 'str'


In [ ]:
import numpy as np
import tensorflow as tf
from sklearn.metrics import classification_report, confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import pandas as pd

# Завантаження тестових даних
test_data = pd.read_csv('test_data.csv')

input_ids = np.array(test_data['input_ids'].apply(lambda x: np.fromstring(x.strip("[]"), sep=',')).tolist())
attention_mask = np.array(test_data['attention_mask'].apply(lambda x: np.fromstring(x.strip("[]"), sep=',')).tolist())

test_labels = test_data.drop(columns=['input_ids', 'attention_mask']).values  # Всі інші стовпці вважаємо мітками

# Завантаження моделі
model = tf.keras.models.load_model('model.h5')

# Прогнозування
predictions = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})

# Перетворення прогнозів у бінарний формат
threshold = 0.5
preds_binary = (predictions > threshold).astype(int)

# Переконайтесь, що мітки також у бінарному форматі
test_labels_binary = np.array(test_labels)

# Класифікаційний звіт
print("Classification Report:\n", classification_report(test_labels_binary, preds_binary))

In [ ]:
for i, class_name in enumerate(['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']):
    cm = confusion_matrix(test_labels_binary[:, i], preds_binary[:, i])
    disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=[f"not_{class_name}", class_name])
    disp.plot(cmap=plt.cm.Blues)
    plt.title(f"Confusion Matrix for {class_name}")
    plt.show()

In [ ]:
history = model.history
if history:
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.ylabel('Accuracy')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.show()

    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.ylabel('Loss')
    plt.xlabel('Epoch')
    plt.legend(loc='best')
    plt.show()